# Recurrent neural network

Character-level language model

<img src="http://karpathy.github.io/assets/rnn/charseq.jpeg">

In [1]:
using Flux
using Flux: onehot, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition

## Download data

In [2]:
isfile("input.txt") ||
  download("http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt", "input.txt")

true

## Preprocessing

In [3]:
text = collect(String(read("input.txt")))
alphabet = [unique(text)..., '_']
text = map(ch -> onehot(ch, alphabet), text)
stop = onehot('_', alphabet);

In [4]:
N = length(alphabet)
seqlen = 50
nbatch = 50

50

In [5]:
Xs = collect(partition(batchseq(chunk(text, nbatch), stop), seqlen))
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch), stop), seqlen))

1830-element Array{Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},1}:
 [[false false … false false; true false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; true false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false true; … ; false false … false false; false false … false false], [false fals

## Model

In [6]:
m = Chain(
    LSTM(N, 128),
    LSTM(128, 128),
    Dense(128, N),
    softmax)

Chain(Recur(LSTMCell(68, 128)), Recur(LSTMCell(128, 128)), Dense(128, 68), NNlib.softmax)

## Loss function

In [7]:
function loss(xs, ys)
  l = sum(crossentropy.(m.(xs), ys))
  Flux.truncate!(m)
  return l
end

loss (generic function with 1 method)

## Optimizer

In [8]:
opt = ADAM(0.01)
evalcb() = @show loss(Xs[5], Ys[5])

evalcb (generic function with 1 method)

## Training

In [9]:
Flux.train!(loss, params(m), zip(Xs, Ys), opt, cb=throttle(evalcb, 30))

loss(Xs[5], Ys[5]) = 198.97809f0 (tracked)
loss(Xs[5], Ys[5]) = 168.38959f0 (tracked)
loss(Xs[5], Ys[5]) = 166.91396f0 (tracked)
loss(Xs[5], Ys[5]) = 164.40529f0 (tracked)
loss(Xs[5], Ys[5]) = 155.04341f0 (tracked)
loss(Xs[5], Ys[5]) = 141.97955f0 (tracked)
loss(Xs[5], Ys[5]) = 130.9381f0 (tracked)
loss(Xs[5], Ys[5]) = 125.90676f0 (tracked)
loss(Xs[5], Ys[5]) = 121.039375f0 (tracked)
loss(Xs[5], Ys[5]) = 117.26552f0 (tracked)
loss(Xs[5], Ys[5]) = 115.39844f0 (tracked)
loss(Xs[5], Ys[5]) = 112.54724f0 (tracked)
loss(Xs[5], Ys[5]) = 109.97373f0 (tracked)
loss(Xs[5], Ys[5]) = 108.18393f0 (tracked)
loss(Xs[5], Ys[5]) = 107.058525f0 (tracked)
loss(Xs[5], Ys[5]) = 105.0179f0 (tracked)
loss(Xs[5], Ys[5]) = 104.89046f0 (tracked)
loss(Xs[5], Ys[5]) = 103.12332f0 (tracked)
loss(Xs[5], Ys[5]) = 102.071915f0 (tracked)
loss(Xs[5], Ys[5]) = 101.54724f0 (tracked)
loss(Xs[5], Ys[5]) = 101.81197f0 (tracked)
loss(Xs[5], Ys[5]) = 99.7125f0 (tracked)
loss(Xs[5], Ys[5]) = 99.860146f0 (tracked)
loss(Xs[5], 

## Sampling

In [10]:
function sample(m, alphabet, len; temp=1)
    Flux.reset!(m)
    buf = IOBuffer()
    c = rand(alphabet)
    for i = 1:len
        write(buf, c)
        c = wsample(alphabet, m(onehot(c, alphabet)).data)
    end
    return String(take!(buf))
end

sample (generic function with 1 method)

In [11]:
sample(m, alphabet, 1000) |> println

hse the merce, to the all gualy
Fear'd'st me.

DUMAIN:
What, old witnd:.

FERDINAND:
What, my giagle, and the angraves, beils angrallials:
And when if fill, thy engra friends:
Read me leed holden our face
By the mester to ter,: Necvenant he bed!
For, seek'd or strangre shape's his lord;
Thou abstard? Ho, usals, but doneom to a learght,
Is if he he rest me cand sufforus,--

Lastleman:
See I shall purse of this adader me!

Second Gin: I will not be calliged of seemer?

PAcERINA:
And name, my deared, to make-fast me means.

IMOGEN:
Ha you renience: quittings, in might marry.
We have mewn man wife his sight,
For the part very believe,-let thee, refo colxing.

WARWICK:
His witnon present suffer, thou somemiep!

Seconat three heer:
Then, let me?

PATROCLUS:
How is we anger.

BIRON:
For the words!' 'tis charnem'd, to paleman.

PANDO:
Let's peace Follows, at Voofsious; no knee extrews?

DON ADRIANO DE ARMADO:
Makes me:
When I will well must limberment?

WROT:
How do; I had, my most: Think nobl